In [0]:
neg_labels = np.array(SAMPLE_SIZE * [[0,1]])
neg_labels

http://ai.stanford.edu/~amaas/data/sentiment/


In [0]:
!pip install nltk
!pip install --upgrade gensim

import numpy as np
import os
import os.path

from nltk.tokenize import word_tokenize
nltk.download('punkt')

if not os.path.isfile('aclImdb_v1.tar.gz'):
  !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

if not os.path.isfile('aclImdb'):  
  !tar -xf aclImdb_v1.tar.gz 
  

In [0]:
!tail aclImdb/imdb.vocab

In [0]:
import glob

SAMPLE_SIZE=600
positive_sample_file_list = glob.glob(os.path.join('aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_sample_file_list[:SAMPLE_SIZE]

negative_sample_file_list = glob.glob(os.path.join('aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_sample_file_list[:SAMPLE_SIZE]


# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


In [0]:
pos_strings = [load_doc(x) for x in positive_sample_file_list]
print(pos_strings[:10])

neg_strings = [load_doc(x) for x in negative_sample_file_list]
print(neg_strings[:10])
    

In [0]:
pos_labels = np.array(SAMPLE_SIZE * [[1,0]])
pos_labels

In [0]:
pos_tokenized = [word_tokenize(s) for s in pos_strings]
print(pos_tokenized[1])
print(pos_tokenized[2])

In [0]:
neg_tokenized = [word_tokenize(s) for s in neg_strings]
print(neg_tokenized[1])
print(neg_tokenized[2])

In [0]:
# load doc into memory
with open('aclImdb/imdb.vocab') as f:
  content = f.readlines()
universe_vocabulary = [x.strip() for x in content]

print(len(universe_vocabulary))
print(len(set(universe_vocabulary)))

In [121]:
from gensim.models import Word2Vec
model_ted = Word2Vec(sentences=pos_tokenized, size=100, window=5, min_count=5, workers=1, sg=0, seed=42)
model_ted.wv.most_similar("brother")

print(np.linalg.norm(model_ted.wv['man'] - model_ted.wv['woman']))
print(np.linalg.norm(model_ted.wv['father'] - model_ted.wv['mother']))
print(np.linalg.norm(model_ted.wv['brother'] - model_ted.wv['sister']))
print(np.linalg.norm(model_ted.wv['house'] - model_ted.wv['ship']))

print(np.linalg.norm(model_ted.wv['father'] - model_ted.wv['mother']))
print(np.linalg.norm(model_ted.wv['sister'] - model_ted.wv['mother']))

0.80154437
0.512299
1.0145818
1.1860116
0.512299
1.1867262


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
